In [3]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import spacy
from spacy.lang.en import English
from spacy.lang.en.stop_words import STOP_WORDS
import string
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn import metrics

# Data Cleaning

In [4]:
# Create our list of punctuation marks
punctuations = string.punctuation

# Create our list of stopwords
nlp = spacy.load('en_core_web_sm')
stop_words = spacy.lang.en.stop_words.STOP_WORDS

# Load English tokenizer, tagger, parser, NER and word vectors
#parser = English()

In [6]:
def lemmatize(doc):
    return [token.lemma_ for token in doc if not
           (token.is_punct or token.is_space or token.lower_ in STOP_WORDS)]

In [7]:
# Creating our tokenizer function. This function will lemmatize, remove stop words, remove punctuations and remove noun? proper noun?
def spacy_tokenizer(doc):
    # Creating our token object, which is used to create documents with linguistic annotations.
    #mytokens = parser(sentence)
    tokens = nlp(doc)
    #print(tokens)

    # Lemmatizing each token and converting each token into lowercase
    #tokens = [ word.lemma_.lower().strip() if word.lemma_ != "-PRON-" else word.lower_ for word in mytokens ]
    tokens = [word.lemma_.lower() for word in tokens]
    #print(tokens)
    
    # Removing stop words and punctuations
    tokens = [ word for word in tokens if word not in stop_words and word not in punctuations ]

    # return preprocessed list of tokens
    return tokens

In [8]:
sent = 'Hey my name is Roberto and I am the best!'
doc = nlp(sent)
lemmatize(doc)

['hey', 'Roberto', 'good']

In [9]:
spacy_tokenizer(sent)

['hey', 'roberto', 'good']

# Modeling

In [11]:
from sklearn.base import TransformerMixin 

# This function will clean the text
def clean_text(text):     
    return text.strip().lower()
    
#Custom transformer using Python standard library (you could use spacy as well)
class predictors(TransformerMixin):

    def transform(self, X, **transform_params):
        return [clean_text(text) for text in X]

    def fit(self, X, y=None, **fit_params):
        return self

    def get_params(self, deep=True):
        return {}

In [14]:
pre = predictors().fit_transform(['I do enjoy my job','I feel amazing!'])
pre

['i do enjoy my job', 'i feel amazing!']

In [11]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer

# We create our bag of words (bow) using our tokenizer and defining an ngram range
bow = CountVectorizer(tokenizer = spacy_tokenizer, ngram_range=(1,1)) 
# Using Tfidf
tfvectorizer = TfidfVectorizer(tokenizer = spacy_tokenizer)

In [ ]:
# Features and Labels
X = df['Message']
ylabels = df['Target']

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, ylabels, test_size=0.3, random_state=42)

In [ ]:
tf = tfvectorizer.fit_transform(X_train)

In [ ]:
tf.toarray()

In [ ]:
pipe = Pipeline([("cleaner", predictors()),
                 ('vectorizer', tfvectorizer)], verbose=True)

In [ ]:
example = ["I do enjoy my job",
 "What a poor product!,I will have to get a new one",
 "I feel amazing!",
 "This class sucks"]

In [ ]:
res = pipe.fit_transform(example)

In [ ]:
res.toarray()

In [ ]:
#Function to plot confusion matrix.
def cmplot(cm):
    ax= plt.subplot()
    sns.heatmap(cm, annot=True, fmt='g', ax=ax);  #annot=True to annotate cells, ftm='g' to disable scientific notation

    # labels, title and ticks
    ax.set_xlabel('Predicted labels');
    ax.set_ylabel('True labels'); 
    ax.set_title('Confusion Matrix'); 
    ax.xaxis.set_ticklabels(['0', '1']); 
    ax.yaxis.set_ticklabels(['0', '1']);

## Logistic Regression

In [ ]:
# Logistic Regression Classifier
from sklearn.linear_model import LogisticRegression
classifier_LG = LogisticRegression(verbose=True)

In [ ]:
# Create the  pipeline to clean, tokenize, vectorize, and classify 
pipe1_LG = Pipeline([("cleaner", predictors()),
                 ('vectorizer', bow),
                 ('classifier', classifier_LG)], verbose=True)

pipe2_LG = Pipeline([("cleaner", predictors()),
                 ('vectorizer', tfvectorizer),
                 ('classifier', classifier_LG)], verbose=True)

In [ ]:
pipe1_LG.fit(X_train,y_train)
pipe2_LG.fit(X_train,y_train)

In [ ]:
plg1_prediction = pipe1_LG.predict(X_test)
plg2_prediction = pipe2_LG.predict(X_test)

In [ ]:
metrics.confusion_matrix(y_test, plg1_prediction)

In [ ]:
cm=metrics.confusion_matrix(y_test, plg1_prediction)
cmplot(cm)

In [ ]:
print("Accuracy:",metrics.accuracy_score(y_test, plg1_prediction))
print("Precision:",metrics.precision_score(y_test, plg1_prediction))
print("Recall:",metrics.recall_score(y_test, plg1_prediction))
print("Specificity:", metrics.recall_score(y_test, plg1_prediction,pos_label=0))
print("F1 Score: ",metrics.f1_score(y_test,plg1_prediction))

In [ ]:
metrics.roc_auc_score(y_test, plg1_prediction)

In [ ]:
metrics.confusion_matrix(y_test, plg2_prediction)

In [ ]:
cm=metrics.confusion_matrix(y_test, plg2_prediction)
cmplot(cm)

In [ ]:
print("Accuracy:",metrics.accuracy_score(y_test, plg2_prediction))
print("Precision:",metrics.precision_score(y_test, plg2_prediction))
print("Recall:",metrics.recall_score(y_test, plg2_prediction))
print("Specificity:", metrics.recall_score(y_test, plg2_prediction,pos_label=0))
print("F1 Score: ",metrics.f1_score(y_test,plg2_prediction))

In [ ]:
metrics.roc_curve(y_test, plg2_prediction)

In [ ]:
metrics.roc_auc_score(y_test, plg2_prediction)

In [ ]:
from sklearn.model_selection import cross_val_score

In [ ]:
scores = cross_val_score(pipe2_LG, X, ylabels,cv=2)

In [ ]:
scores

In [ ]:
example = ["I do enjoy my job",
 "What a poor product!,I will have to get a new one",
 "I feel amazing!",
 "This class sucks"]

pipe2_LG.predict(example)

In [ ]:
pipe2_LG.decision_function
pipe2_LG.predict_proba(example)

In [ ]:
pipe2_LG
pipe2_LG.decision_function
pipe2_LG.steps[2][1].coef_

In [ ]:
pipe2_LG.steps[2][1].coef_.shape
pipe2_LG.steps[1][1].vocabulary_#3270

# Load model

In [2]:
import joblib
model = joblib.load("model/logreg_tfidf.pkl")

In [3]:
sent = ['This product is awesome!']
model.predict(sent)

array([1])

In [4]:
model.predict_proba(sent)

array([[0.18719386, 0.81280614]])

In [3]:
model.steps[1][1].vocabulary_.keys()

dict_keys(['product', 'clearly', 'ready', 'prime', 'time', 'care', 'cool', 'look', 'tell', 'come', 'worthless', 'price', 'great', 'deal', 'work', 'weekly', 'haunt', 'definitely', 'place', 'waiter', 'jerk', 'tragedy', 'strike', 'love', 'hit', 'spot', 'want', 'healthy', 'lack', 'quantity', 'flavor', 'week', 'later', 'activate', 'suddenly', 'die', 'keypad', 'tinny', 'reach', 'wrong', 'button', 'order', 'lemon', 'raspberry', 'ice', 'cocktail', 'incredible', 'probably', 'wo', 'interesting', 'action', 'suspense', 'unneeded', 'controversy', 'hold', 'charge', 'long', 'reasonably', 'comfortable', 'wear', 'condition', 'quality', 'sound', 'tremendous', 'bad', 'customer', 'service', 'terrible', 'commercial', 'misleading', 'involve', 'return', 'school', 'acting', 'utterly', 'predictable', 'script', 'pile', 'garbage', 'round', 'buy', 'brain', 'attempt', 'shut', 'primal', 'impulse', 'self', 'preservation', 'share', 'general', 'loewenhielm', 'exquisite', 'joy', 'partaking', 'caille', 'en', 'sarcophage

In [52]:
model.steps[1][1].vocabulary_.get('......')

12

In [6]:
model.steps[2][1].coef_[0][291]
model.steps[2][1].coef_[0]

array([ 0.05118546, -0.0124722 ,  0.039356  , ..., -0.17987326,
        0.13392557, -0.056772  ])

In [43]:
len(model.steps[1][1].get_feature_names())
model.steps[1][1].get_feature_names()
model.steps[1][1].vocabulary_.get('bad')

291

In [39]:
model.steps[1][1].get_feature_names()[291]

'bad'

In [40]:
model.steps[1][1]

TfidfVectorizer(tokenizer=<bound method SentimentTrain.spacy_tokenizer of <utils.SentimentTrain object at 0x7fc5af177290>>)

In [32]:
model.steps[1][1].vocabulary_
for key, val in enumerate(model.steps[1][1].vocabulary_.items()):
    print(key, val[0], val[1])
    

0 product 2233
1 clearly 599
2 ready 2314
3 prime 2222
4 time 2922
5 care 499
6 cool 700
7 look 1723
8 tell 2877
9 come 631
10 worthless 3237
11 price 2217
12 great 1334
13 deal 793
14 work 3232
15 weekly 3185
16 haunt 1380
17 definitely 814
18 place 2144
19 waiter 3149
20 jerk 1594
21 tragedy 2965
22 strike 2766
23 love 1738
24 hit 1416
25 spot 2715
26 want 3154
27 healthy 1388
28 lack 1645
29 quantity 2278
30 flavor 1176
31 week 3183
32 later 1661
33 activate 127
34 suddenly 2795
35 die 856
36 keypad 1624
37 tinny 2927
38 reach 2310
39 wrong 3244
40 button 465
41 order 2009
42 lemon 1681
43 raspberry 2300
44 ice 1475
45 cocktail 615
46 incredible 1516
47 probably 2226
48 wo 3221
49 interesting 1559
50 action 126
51 suspense 2832
52 unneeded 3054
53 controversy 690
54 hold 1421
55 charge 545
56 long 1722
57 reasonably 2322
58 comfortable 635
59 wear 3177
60 condition 664
61 quality 2277
62 sound 2687
63 tremendous 2978
64 bad 291
65 customer 767
66 service 2557
67 terrible 2884
68 com

1061 spec 2702
1062 wow 3239
1063 expert 1094
1064 connisseur 672
1065 topic 2949
1066 experienced 1093
1067 frenchman 1229
1068 debate 795
1069 sack 2460
1070 trumpeter 2992
1071 falsely 1119
1072 accuse 118
1073 murder 1903
1074 pure 2270
1075 horror 1442
1076 stupid 2776
1077 exercise 1084
1078 frustration 1243
1079 father 1135
1080 v265 3087
1081 sorry 2683
1082 ordeal 2008
1083 matter 1798
1084 progress 2238
1085 performance 2107
1086 anguish 192
1087 suffering 2797
1088 glass 1303
1089 situations:1 2630
1090 race 2287
1091 card 496
1092 aside 241
1093 lets 1688
1094 major 1765
1095 destroy 844
1096 latifa 1662
1097 suffer 2796
1098 hard 1375
1099 premise 2206
1100 president 2212
1101 u.s 3011
1102 allow 170
1103 recently 2329
1104 witness 3219
1105 guest 1353
1106 30 55
1107 min 1850
1108 pancake 2059
1109 egg 996
1110 rock 2437
1111 casino 514
1112 forward 1221
1113 access 106
1114 download 919
1115 ringtone 2425
1116 jerky 1595
1117 movement 1891
1118 overall 2030
1119 scale 24

1548 text 2888
1549 tricky 2981
1550 remarkable 2371
1551 animation 194
1552 tool 2947
1553 milkshake 1849
1554 chocolate 577
1555 milk 1848
1556 loose 1726
1557 connection 671
1558 couple 715
1559 limit 1701
1560 crab 726
1561 leg 1679
1562 sunday 2808
1563 ago 156
1564 march 1780
1565 20th 49
1566 2005 46
1567 entire 1039
1568 certain 534
1569 11 25
1570 12 27
1571 bored 402
1572 jessice 1597
1573 lange 1652
1574 clothe 610
1575 jalapeno 1583
1576 soooo 2677
1577 shelf 2580
1578 uncomfortable 3022
1579 compare 645
1580 wire 3214
1581 lg 1690
1582 10/10 23
1583 betty 352
1584 jean 1591
1585 single 2623
1586 employee 1016
1587 ok 1986
1588 water 3167
1589 refill 2345
1590 finally 1160
1591 rare 2298
1592 instance 1544
1593 trek 2977
1594 v 3086
1595 final 1159
1596 frontier 1239
1597 series 2552
1598 defective 811
1599 crowe 751
1600 sub 2782
1601 attention 259
1602 ignore 1484
1603 laptop 1654
1604 ir 1572
1605 subject 2783
1606 jay 1589
1607 adams 134
1608 unfortunate 3039
1609 neil 

2075 consumer 683
2076 errol 1049
2077 flynn 1197
2078 dad 774
2079 favourite 1139
2080 grow 1348
2081 zillion 3264
2082 neighbour 1929
2083 misplace 1860
2084 main 1761
2085 excuse 1082
2086 poorly 2180
2087 trunk 2993
2088 hitch 1417
2089 originally 2015
2090 discard 883
2091 killer 1630
2092 chai 538
2093 latte 1664
2094 terribly 2885
2095 award 273
2096 accolade 113
2097 deserving 836
2098 casting 516
2099 science 2507
2100 teacher 2865
2101 sadly 2463
2102 gordon 1320
2103 ramsey 2291
2104 shall 2568
2105 sharply 2574
2106 emptiness 1017
2107 atmosphere.1 252
2108 fair 1115
2109 11.99 26
2110 subway 2791
2111 dvd 957
2112 halfway 1361
2113 embarrassed 1009
2114 howell 1452
2115 sprint 2717
2116 rarely 2299
2117 note 1955
2118 chipotle 576
2119 interior 1561
2120 latch 1659
2121 endearing 1022
2122 echo 979
2123 unsatisfactory 3061
2124 cutout 772
2125 pitiful 2141
2126 respect 2393
2127 law 1667
2128 mall 1768
2129 grandmother 1327
2130 ideal 1478
2131 sensitive 2545
2132 overpric

2828 1947 33
2829 masterpiece 1794
2830 screenplay 2516
2831 post 2192
2832 2mp 52
2833 pic 2132
2834 5(of 68
2835 thunderbird 2918
2836 startac 2729
2837 apologize 212
2838 earphone 968
2839 owl 2040
2840 sos 2685
2841 kathy 1620
2842 bate 320
2843 desperation 839
2844 escapism 1051
2845 variation 3096
2846 field 1150
2847 nigiri 1940
2848 stunning 2775
2849 screamy 2513
2850 masculine 1790
2851 strawberry 2761
2852 attach 256
2853 gas 1275
2854 buyit 468
2855 jessica 1596
2856 entertainment 1038
2857 communication 640
2858 management.oh 1773
2859 activesync 128
2860 4.2 62
2861 optimal 2006
2862 synchronization 2842
2863 ericsson 1048
2864 w810i 3145
2865 drawback 931
2866 mp3 1894
2867 pause 2083
2868 cuisine 757
2869 scream 2512
2870 somethat 2671
2871 whistle 3193
2872 i. 1470
2873 madhouse 1756
2874 ache 119
2875 proclaim 2230
2876 wildly 3204
2877 caterpillar 520
2878 flair 1171
2879 hummh 1458
2880 ms 1896
2881 angry 191
2882 cumbersome 760
2883 ineptly 1528
2884 stereotypicall

In [42]:
for i,token in enumerate(model.steps[1][1].get_feature_names()):
    print(i,token, model.steps[2][1].coef_[0][i])
    

0 
 0.05118546316120706
1 
  -0.01247220137867434
2  	 0.03935599931795044
3 (; 0.1412662107892729
4 -)oh 0.214858974894566
5 -- 0.33204656291005236
6 -drink -0.11077081208787312
7 -my -0.25228550676324313
8 .. -0.21051547979536292
9 ... 0.11963600130947302
10 .... -0.6592642921834981
11 ..... 0.30669049887285227
12 ...... -0.12435698441347195
13 ............. 0.11603219755295267
14 .i 0.0955752883632471
15 0 0.07019340666062789
16 0/10 -0.21048353242203244
17 1 -0.6432131751388571
18 1.3 -0.11749578931035522
19 1.5 -0.11335961944295997
20 1/10 -0.21048353242203244
21 1/2 -0.1864843957173609
22 10 1.0560078140707345
23 10/10 0.7459098296028732
24 100 0.0031160885238038583
25 11 -0.10194720480406859
26 11.99 -0.11410403565251237
27 12 -0.33263692856340143
28 13 -0.16550991000894325
29 15 0.5252938365849282
30 17 -0.08569257824827117
31 18th 0.10103650943502486
32 1928 0.13976228793859508
33 1947 0.19966017008420067
34 1948 -0.12179661507257108
35 1949 0.11187140550041832
36 1971 -0.1120

531 center 0.006542581250074474
532 central 0.00941937926644315
533 century -0.019690262493792746
534 certain -0.3295604641698627
535 certainly -0.06885196737524255
536 cg -0.12786550601959937
537 cgi 0.08580509437382221
538 chai 0.25117389192999195
539 challenge 0.07663543381795065
540 chance 0.11376441651655395
541 change -0.29104587426901607
542 char -0.2406193251690365
543 character 0.7949024638639823
544 charcoal -0.07407423592644226
545 charge -0.9528862830602852
546 charger -0.17460134539984237
547 charisma -0.34110480189555026
548 charismatic 0.24357704661889026
549 charlie 0.11811667282622153
550 charm 0.5196151752726567
551 charming 0.14498798324087103
552 chase 0.006542581250074474
553 cheap -0.23310850725174043
554 cheaply -0.4026840020596468
555 cheapy 0.20940443480010834
556 cheat -0.16282087406645857
557 check 0.5887018945606175
558 cheek 0.24059645787839687
559 cheekbone 0.07289446329671488
560 cheerfull 0.13169041958803038
561 cheese -0.06541918641115473
562 cheeseburg

1015 emperor -0.15159234966487203
1016 employee -0.28776597849633473
1017 emptiness -0.2044772022219043
1018 en 0.10981416857381222
1019 enchanting 0.08580509437382221
1020 encourage 0.19804813804347682
1021 end -0.018968912027809686
1022 endearing 0.1296747673185258
1023 ending 0.21965084530896525
1024 endlessly -0.1519003238967956
1025 energetic -0.15128744429588162
1026 energy -0.07331147540056253
1027 engage -0.25309604332788876
1028 engaging 0.13533257660929685
1029 engineer 0.13872826604207697
1030 english -0.21241643715281985
1031 enjoy 1.3116026235090827
1032 enjoyable 0.2812877604648773
1033 enjoyment 0.20699213279216394
1034 ensue -0.12253457928626887
1035 enterprise -0.0821380223735036
1036 entertain 0.29823184790187174
1037 entertaining 0.6556185070197619
1038 entertainment 0.08908267756189261
1039 entire 0.3943716101010175
1040 entirely -0.2132099694547489
1041 entrance 0.13145291823196406
1042 entree -0.15485527749315678
1043 episode 0.2717206730155109
1044 equally 0.1134

1653 lap -0.1969489009980319
1654 laptop 0.25668778539491327
1655 large 0.44132830776926635
1656 largely -0.1473192336277021
1657 laselva 0.006542581250074474
1658 lassie -0.1948582915557606
1659 latch -0.007020962904692166
1660 late 0.20031861569662832
1661 later -0.4558988495823841
1662 latifa -0.1412983879314929
1663 latin -0.05599440871847323
1664 latte 0.25117389192999195
1665 laugh 0.2784359990648885
1666 laughable -0.07493560228037907
1667 law -0.13379772885026803
1668 layer 0.07381251520194465
1669 lb -0.11335961944295997
1670 lead -0.09875789551149315
1671 leaf -0.094395111701757
1672 leak -0.19268214244761434
1673 leap 0.11158139918488702
1674 learn 0.07243255755071554
1675 leather 0.26901209028718664
1676 leave -0.4365663088146353
1677 lee 0.08028619070096872
1678 leftover -0.14743957355791895
1679 leg -0.2534455329133178
1680 legit 0.2803192982421172
1681 lemon 0.16544130060183
1682 length 0.13747111173737428
1683 leni 0.006542581250074474
1684 leopard 0.2197131726541201
16

2320 reason -0.5298885036836987
2321 reasonable 0.09394434168511873
2322 reasonably 0.6386106146766765
2323 reboots.overall 0.1145212360064705
2324 recall -0.19433377351925646
2325 reccomendation 0.24374663862084414
2326 receipt -0.103425346124194
2327 receive -0.613994198973693
2328 recent -0.024111685744241614
2329 recently -0.10229562580778928
2330 reception 0.2913462570812667
2331 reception.a 0.061631217320774985
2332 recessed -0.091594595607181
2333 recharge -0.21879058349624617
2334 recieve -0.12435698441347195
2335 recognition 0.2627852253103881
2336 recognize -0.27717986526724936
2337 recommend 0.6351930202493409
2338 recommendation 0.2522148726883132
2339 reconciliation 0.15996380548490605
2340 recover -0.2007743731264505
2341 recur 0.07817731968037481
2342 red -0.07158408035231495
2343 redeem -0.3474229084189935
2344 reference 0.006542581250074474
2345 refill -0.12819935494136248
2346 refrain -0.15334593459840096
2347 refreshing 0.08580509437382221
2348 refried -0.09484298754

2670 solidify -0.10100965352903679
2671 somethat 0.17964185271217406
2672 somewhat -0.13630995694010656
2673 son 0.08278100041816806
2674 song 0.17475977843182797
2675 sony 0.06630051090025713
2676 soon 0.28439590802582926
2677 soooo 0.3222219185860246
2678 sooooo 0.2326672926590963
2679 soooooo -0.14290726917475824
2680 sophisticated 0.006542581250074474
2681 sorely -0.1358586273612348
2682 sorrentino 0.09311022360303757
2683 sorry -0.7261929713763321
2684 sort 0.30239187502415843
2685 sos 0.1643119386997381
2686 soul 0.19966017008420067
2687 sound 0.5895368305787817
2688 soundtrack 0.1451140791486592
2689 soup 0.0796420522389533
2690 sour 0.20410056411152933
2691 source -0.09551784794873185
2692 south 0.29802500499955753
2693 southern 0.12434477200126773
2694 soyo -0.16882667280384966
2695 space -0.14728310756064902
2696 spacey 0.006542581250074474
2697 spaghetti -0.20953716056269275
2698 span 0.23975794019437335
2699 speak -0.14003209581211923
2700 speaker 0.14758805618599882
2701 s

3253 yellowtail 0.17304533675462883
3254 yelper -0.10136335583797104
3255 yelps 0.0662870053291526
3256 yes -0.1935736408047194
3257 young 0.10173581649674905
3258 youtube -0.3827601312493151
3259 yucky -0.12591807081041323
3260 yum -0.218654887543937
3261 yummy 0.32506336660513063
3262 z500a 0.15101063627767145
3263 zero -0.4715257697862745
3264 zillion -0.2077635390311292
3265 zombie 0.006542581250074474
3266 zombiez -0.15382774177584155
3267  -0.17987326078489482
3268   0.13392556520017687
3269  -0.056771999446520185


In [36]:
model.steps[1][1].vocabulary_.get('cool')

700

In [38]:
model.steps[2][1].coef_[0][291]
#model.steps[1][1].vocabulary_[29]

-3.5514173914594664

In [19]:
for i,coef in enumerate(model.steps[2][1].coef_[0]):
    print(i, coef)
    

0 0.05118546316120706
1 -0.01247220137867434
2 0.03935599931795044
3 0.1412662107892729
4 0.214858974894566
5 0.33204656291005236
6 -0.11077081208787312
7 -0.25228550676324313
8 -0.21051547979536292
9 0.11963600130947302
10 -0.6592642921834981
11 0.30669049887285227
12 -0.12435698441347195
13 0.11603219755295267
14 0.0955752883632471
15 0.07019340666062789
16 -0.21048353242203244
17 -0.6432131751388571
18 -0.11749578931035522
19 -0.11335961944295997
20 -0.21048353242203244
21 -0.1864843957173609
22 1.0560078140707345
23 0.7459098296028732
24 0.0031160885238038583
25 -0.10194720480406859
26 -0.11410403565251237
27 -0.33263692856340143
28 -0.16550991000894325
29 0.5252938365849282
30 -0.08569257824827117
31 0.10103650943502486
32 0.13976228793859508
33 0.19966017008420067
34 -0.12179661507257108
35 0.11187140550041832
36 -0.11204816018192854
37 -0.18498760797487346
38 0.10500157537194804
39 -0.14903733967342186
40 -0.12344368834304403
41 -0.12331516227386191
42 -0.21297298396851477
43 0.

807 0.006542581250074474
808 -0.21015289895488
809 0.10358699194209575
810 0.29482102272845917
811 -0.410407302428152
812 -0.01247220137867434
813 0.18736580606666395
814 1.198682092399627
815 0.3415641132993908
816 0.23823628855472886
817 -0.12339639742839688
818 -0.1465779850760686
819 -0.2870128110580579
820 0.21080241867088137
821 0.31330439118979414
822 2.041173878460134
823 0.1539864588133845
824 0.4543464934065934
825 0.31934240181593737
826 0.11452804559393244
827 0.44758730329024815
828 0.28061922626451313
829 0.14001112714107894
830 -0.19101081783714782
831 -0.3299906718879286
832 0.12068290121813521
833 -0.3382255976943495
834 0.12434477200126773
835 -0.457972438239606
836 -0.03476802727894036
837 -0.5205580820448468
838 0.2643093598185928
839 0.08391329153527498
840 -0.1387007903939572
841 -0.23287369296907534
842 0.011228581000857947
843 0.24816926740506273
844 -0.03497695815267738
845 0.17438288057579399
846 0.21906541692684817
847 -0.10100965352903679
848 0.6669340500251

1676 -0.4365663088146353
1677 0.08028619070096872
1678 -0.14743957355791895
1679 -0.2534455329133178
1680 0.2803192982421172
1681 0.16544130060183
1682 0.13747111173737428
1683 0.006542581250074474
1684 0.2197131726541201
1685 -0.10553571700251493
1686 -0.1637133814160657
1687 -0.14429939688498405
1688 -0.1412983879314929
1689 0.025482185031996618
1690 0.0219215161809719
1691 0.0662870053291526
1692 0.5022610800506884
1693 0.2202560641554373
1694 0.38729593765980336
1695 -0.19834550100794748
1696 0.2672280937656355
1697 0.40459385114499147
1698 0.75046829720011
1699 0.08028619070096872
1700 0.19509380555935535
1701 -0.1267227664566589
1702 -0.21826552980580832
1703 0.006542581250074474
1704 -0.329370197568743
1705 0.2167822567488447
1706 0.4688176627101934
1707 -0.07799311432841885
1708 0.3413166002909014
1709 0.07880999056795338
1710 -0.2877590702602328
1711 0.08838343711682888
1712 -0.0943336142133609
1713 -0.41099693994866543
1714 -0.3246355638853199
1715 -0.09404212884015267
1716 0

2234 -0.0028490829996314308
2235 0.3280379910244893
2236 -0.10100965352903679
2237 0.08899202755833202
2238 0.05537418825682684
2239 0.22321550120212647
2240 0.40672579194335706
2241 0.45601780908117895
2242 0.09762631507456845
2243 -0.10666368679961331
2244 -0.3351948977712226
2245 0.0956561595057298
2246 0.11086468071533129
2247 0.19331879273970665
2248 0.10442100519682791
2249 0.12409418724099272
2250 0.02658963543618533
2251 0.10887822220618895
2252 0.1024678715683083
2253 0.38818323935460824
2254 0.05966303816825918
2255 0.33752268590661677
2256 0.1657847627001565
2257 0.2498091521273017
2258 -0.05599440871847323
2259 0.12436386942883289
2260 0.1848008789027786
2261 -0.01295415958884883
2262 -0.1707087452389673
2263 -0.08780541504987482
2264 0.09431310407328967
2265 0.21648479259222483
2266 -0.10458351124192766
2267 -0.26500616321758536
2268 0.2989902861513202
2269 0.10525225414642772
2270 -0.20290877886673955
2271 0.1706880219424447
2272 0.15404495562101977
2273 -0.19309351848841

2935 0.08242251603799726
2936 -0.16796338137365466
2937 0.2968481245836742
2938 -0.11519242952428728
2939 -0.12101052982326024
2940 0.34453573117984093
2941 -0.13570570513446512
2942 0.13976228793859508
2943 -0.41600984280948095
2944 0.24059645787839687
2945 -0.13754140998759973
2946 0.09311022360303757
2947 0.5914392417044771
2948 0.09457003846798978
2949 0.16475072132154497
2950 -0.60869388080891
2951 0.14418612734355862
2952 -0.010279325988154583
2953 -0.36522448078561
2954 0.05646916989417771
2955 0.22568225682489645
2956 -0.11237744204359677
2957 0.11853527074009568
2958 0.6111168160824528
2959 -0.15427680495183446
2960 0.1636898376509423
2961 -0.1685906136478711
2962 0.10643956208898624
2963 -0.10937315131992387
2964 -0.18482585368917684
2965 -0.24084202786375125
2966 0.05911647351357054
2967 0.13133858701718665
2968 0.10743292597844405
2969 0.29714489016954276
2970 0.1352102014493496
2971 0.14090250346842476
2972 0.16469565641638992
2973 -0.3497792217188815
2974 0.16286454819549